In [5]:
import os
import numpy as np
from tqdm import tqdm
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder


In [6]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [7]:
# Cell 2
DATA_PATH = "./Data/Clothes_Dataset"
IMG_SIZE = 64

X = []
y = []

classes = os.listdir(DATA_PATH)

for label in classes:
    folder = os.path.join(DATA_PATH, label)
    for img_name in tqdm(os.listdir(folder), desc=f"Loading {label}"):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype(np.float32) / 255.0  # normalization
        X.append(img.flatten())               # flatten here
        y.append(label)

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)


Loading Sweter: 100%|██████████| 500/500 [00:03<00:00, 132.96it/s]


(7500, 12288) (7500,)


In [8]:
from sklearn.model_selection import train_test_split

le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

In [10]:
class ClothesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_ds = ClothesDataset(X_train, y_train)
test_ds  = ClothesDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=64, shuffle=False)


In [11]:
def get_activation(name):
    if name == "relu":
        return nn.ReLU()
    elif name == "sigmoid":
        return nn.Sigmoid()
    elif name == "tanh":
        return nn.Tanh()

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_layers, activation, num_classes):
        super().__init__()
        layers = []
        prev_dim = input_dim

        for h in hidden_layers:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(get_activation(activation))
            prev_dim = h

        layers.append(nn.Linear(prev_dim, num_classes))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


In [12]:
def train_model(model, optimizer, criterion, epochs=10):
    model.train()
    for _ in range(epochs):
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(Xb), yb)
            loss.backward()
            optimizer.step()

def evaluate_model(model):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for Xb, yb in test_loader:
            Xb = Xb.to(device)
            out = model(Xb)
            preds.extend(out.argmax(1).cpu().numpy())
            trues.extend(yb.numpy())

    return f1_score(trues, preds, average="macro")


In [13]:
input_dim = X.shape[1]

hidden_configs = {
    "1_layer": [256],
    "2_layers": [256, 128],
    "3_layers": [256, 128, 64]
}

activations = ["relu", "sigmoid", "tanh"]

results = []

for act in activations:
    for name, hidden in hidden_configs.items():
        model = MLP(input_dim, hidden, act, num_classes).to(device)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        train_model(model, optimizer, criterion, epochs=10)
        f1 = evaluate_model(model)

        results.append((name, act, f1))
        print(f"{name} | {act} | Macro F1: {f1:.4f}")


1_layer | relu | Macro F1: 0.3309
2_layers | relu | Macro F1: 0.3518
3_layers | relu | Macro F1: 0.3250
1_layer | sigmoid | Macro F1: 0.1783
2_layers | sigmoid | Macro F1: 0.1273
3_layers | sigmoid | Macro F1: 0.0411
1_layer | tanh | Macro F1: 0.0582
2_layers | tanh | Macro F1: 0.0596
3_layers | tanh | Macro F1: 0.0485


In [14]:
import pandas as pd

df = pd.DataFrame(results, columns=["Hidden Layers", "Activation", "Macro F1"])
df


,Hidden Layers,Activation,Macro F1
0,1_layer,relu,0.330901
1,2_layers,relu,0.351835
2,3_layers,relu,0.324956
3,1_layer,sigmoid,0.178320
4,2_layers,sigmoid,0.127309
5,3_layers,sigmoid,0.041141
6,1_layer,tanh,0.058217
7,2_layers,tanh,0.059618
8,3_layers,tanh,0.048536
